In [2]:
import os
import torch
import numpy as np
import pandas as pd
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Normalize
from flask import Flask, request, render_template, flash, redirect, url_for
import inspect
from huggingface_model_utils import load_model_from_local_path
from datetime import datetime

/home/quoc14/Code/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
aligner = load_model_from_local_path('model/minchul/cvlface_DFA_mobilenet').to(device)
fr_model = load_model_from_local_path('model/minchul/cvlface_adaface_vit_base_webface4m').to(device)

Loaded pretrained aligner from pretrained_model/model.pt
Loaded ViT model
compatible keys in state_dict 281 / 281
Check


<All keys matched successfully>
Loaded pretrained model from pretrained_model/model.pt


In [4]:
# Image normalization and alignment function
def pil_to_input(pil_image, device):
    trans = Compose([ToTensor(), Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
    return trans(pil_image).unsqueeze(0).to(device)

def get_feat(input_tensor, aligner, fr_model, device):
    input_tensor = input_tensor.to(device)
    aligned_x, _, aligned_ldmks, _, _, _ = aligner(input_tensor)
    input_signature = inspect.signature(fr_model.model.net.forward)
    if input_signature.parameters.get('keypoints') is not None:
        feat = fr_model(aligned_x, aligned_ldmks)
    else:
        feat = fr_model(aligned_x)
    return feat


In [5]:
import csv
import torch
import time
import os

# Đếm các trường hợp TP, TN, FP, FN
TP = 0  # True Positive
TN = 0  # True Negative
FP = 0  # False Positive
FN = 0  # False Negative

def evaluate_model(pairs_file, threshold, aligner, fr_model, device):
    global TP, TN, FP, FN
    processed_lines = 0  # Biến đếm số dòng đã xử lý

    with open(pairs_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Bỏ qua dòng đầu tiên (header)

        for row in reader:
            # Loại bỏ các phần tử rỗng
            row = [x for x in row if x.strip()]
            
            processed_lines += 1  # Cập nhật biến đếm số dòng đã xử lý

            if len(row) == 3:  # Match pair (cùng một người)
                name = row[0]
                img_num1 = row[1]
                img_num2 = row[2]

                # Đường dẫn ảnh
                path_img1 = f'/home/quoc14/Code/lfw/lfw-deepfunneled/lfw-deepfunneled/{name}/{name}_{str(img_num1).zfill(4)}.jpg'
                path_img2 = f'/home/quoc14/Code/lfw/lfw-deepfunneled/lfw-deepfunneled/{name}/{name}_{str(img_num2).zfill(4)}.jpg'

                # Tính toán cosine similarity
                cossim = compare_images(path_img1, path_img2, aligner, fr_model, device)

                # Phân loại theo threshold
                if cossim > threshold:
                    TP += 1  # Dự đoán đúng cùng một người (True Positive)
                else:
                    FN += 1  # Dự đoán sai khác người (False Negative)

            elif len(row) == 4:  # Mismatch pair (hai người khác nhau)
                name1 = row[0]
                img_num1 = row[1]
                name2 = row[2]
                img_num2 = row[3]
                
                # Đường dẫn ảnh
                path_img1 = f'/home/quoc14/Code/lfw/lfw-deepfunneled/lfw-deepfunneled/{name1}/{name1}_{str(img_num1).zfill(4)}.jpg'
                path_img2 = f'/home/quoc14/Code/lfw/lfw-deepfunneled/lfw-deepfunneled/{name2}/{name2}_{str(img_num2).zfill(4)}.jpg'

                # Tính toán cosine similarity
                cossim = compare_images(path_img1, path_img2, aligner, fr_model, device)

                # Phân loại theo threshold
                if cossim <= threshold:
                    TN += 1  # Dự đoán đúng là khác người (True Negative)
                else:
                    FP += 1  # Dự đoán sai là cùng một người (False Positive)

            # In ra số dòng đã xử lý
            if processed_lines % 10 == 0:
                print(f'Đã xử lý {processed_lines} dòng')

    # Tính toán các chỉ số
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total > 0 else 0
    FNIR = FN / (TP + FN) if (TP + FN) > 0 else 0
    FPIR = FP / (TN + FP) if (TN + FP) > 0 else 0

    print(f'Tổng số dòng đã xử lý: {processed_lines}')
    return accuracy, FNIR, FPIR

# Hàm tính toán cosine similarity giữa hai ảnh
def compare_images(path_img1, path_img2, aligner, fr_model, device):
    # Mở ảnh
    img1 = Image.open(path_img1)
    img2 = Image.open(path_img2)

    # Chuẩn bị input
    input1 = pil_to_input(img1, device)
    input2 = pil_to_input(img2, device)

    # Căn chỉnh và trích xuất đặc trưng
    aligned_x1, _, aligned_ldmks1, _, _, _ = aligner(input1)
    aligned_x2, _, aligned_ldmks2, _, _, _ = aligner(input2)

    feat1 = get_feat(aligned_x1, aligner, fr_model, device)
    feat2 = get_feat(aligned_x2, aligner, fr_model, device)

    # Tính toán cosine similarity
    cossim = torch.nn.functional.cosine_similarity(feat1, feat2).item()
    return cossim

# Hàm để tính kích thước mô hình
def get_model_size(model_path):
    return os.path.getsize(model_path)



# Gọi hàm đánh giá và ghi vào file CSV
def save_evaluation_to_csv(pairs_file, threshold, aligner, fr_model, device, model_path, output_csv):
    # Đo thời gian chạy
    start_time = time.time()
    
    accuracy, FNIR, FPIR = evaluate_model(pairs_file, threshold, aligner, fr_model, device)

    # Kết thúc quá trình đánh giá
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Kích thước mô hình
    model_size = get_model_size(model_path)

    # Bộ nhớ tiêu thụ

    # Lưu vào file CSV
    with open(output_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([model_path, accuracy, FNIR, FPIR, model_size, elapsed_time])
        print(f'Kết quả đã được lưu vào {output_csv}')

# Gọi hàm đánh giá
pairs_file = '/home/quoc14/Code/lfw/pairs.csv'
model_path = 'model/minchul/cvlface_adaface_vit_base_webface4m'
output_csv = 'evaluation_results_kprpe.csv'
threshold = 0.3  # Ngưỡng cosine similarity để phân biệt match/mismatch

# Lưu kết quả đánh giá
save_evaluation_to_csv(pairs_file, threshold, aligner, fr_model, device, model_path, output_csv)



Đã xử lý 10 dòng
Đã xử lý 20 dòng
Đã xử lý 30 dòng
Đã xử lý 40 dòng
Đã xử lý 50 dòng
Đã xử lý 60 dòng
Đã xử lý 70 dòng
Đã xử lý 80 dòng
Đã xử lý 90 dòng
Đã xử lý 100 dòng
Đã xử lý 110 dòng
Đã xử lý 120 dòng
Đã xử lý 130 dòng
Đã xử lý 140 dòng
Đã xử lý 150 dòng
Đã xử lý 160 dòng
Đã xử lý 170 dòng
Đã xử lý 180 dòng
Đã xử lý 190 dòng
Đã xử lý 200 dòng
Đã xử lý 210 dòng
Đã xử lý 220 dòng
Đã xử lý 230 dòng
Đã xử lý 240 dòng
Đã xử lý 250 dòng
Đã xử lý 260 dòng
Đã xử lý 270 dòng
Đã xử lý 280 dòng
Đã xử lý 290 dòng
Đã xử lý 300 dòng
Đã xử lý 310 dòng
Đã xử lý 320 dòng
Đã xử lý 330 dòng
Đã xử lý 340 dòng
Đã xử lý 350 dòng
Đã xử lý 360 dòng
Đã xử lý 370 dòng
Đã xử lý 380 dòng
Đã xử lý 390 dòng
Đã xử lý 400 dòng
Đã xử lý 410 dòng
Đã xử lý 420 dòng
Đã xử lý 430 dòng
Đã xử lý 440 dòng
Đã xử lý 450 dòng
Đã xử lý 460 dòng
Đã xử lý 470 dòng
Đã xử lý 480 dòng
Đã xử lý 490 dòng
Đã xử lý 500 dòng
Đã xử lý 510 dòng
Đã xử lý 520 dòng
Đã xử lý 530 dòng
Đã xử lý 540 dòng
Đã xử lý 550 dòng
Đã xử lý 560 dòng
Đ

: 